### GIS specific List Comprehension Example

In [ ]:
import arcpy
from arcpy import env

### Set environment

In [ ]:
path = "/path/to/folder/Default.gdb"
env.workspace = path
arcpy.env.overwriteOutput = True

### Create Variables

In [ ]:
bec = "BEC_BIOGEOCLIMATIC_20K_SPG"
frqTbl = "becLUT"
tmpTbl = "tempBecTbl"

### BEC list for analysis

In [ ]:
becList = ['BG','PP','IDF','MS','ESSF','ICH','CWH']

### Print the list

In [ ]:
becList

### To make the BEC table smaller I use `Frequency` tool to sum up the BEC table by `MAP_LABEL`.

In [ ]:
arcpy.Frequency_analysis(bec, frqTbl, "'MAP_LABEL'")

### In order to use the `SelectLayerByAttribute` tool we need to create a temporary table.

In [ ]:
arcpy.MakeTableView_management(frqTbl, tmpTbl)

### If we try to select attributes from the MAP_LABEL field in the BEC table it not select all attributes. Only attributes with the BEC Zones but non of the SubZones and variants.

In [ ]:
for s in becList:
    where_clause = "MAP_LABEL LIKE '"+s+"'"
    arcpy.SelectLayerByAttribute_management(tmpTbl,"ADD_TO_SELECTION",where_clause)
    

results = int(arcpy.GetCount_management(tmpTbl).getOutput(0))
print ("The new selection yielded {0} rows.".format(results))

### This returns zero rows because the `LIKE` operator needs the wildcard (`%`) in order to function and the wildcard is not included in our list. We need to add a wildcard `%` to each item in the list to account for additional characters that represent subzones and variants in the spatial BEC table using the field `MAP_LABEL`.
### This can be done with List Comprehension. In this case List comprehension creates a new list from our existing list
- `x+'%'` is the output function
- `x` is the variable
- `becList` is the input set

In [ ]:
newList = [x+'%' for x in becList] 

### Print the new list

In [ ]:
newList

### Now using the new list with wildcard we can select against the provincial BEC layer.

In [ ]:
for b in newList:
    where_clause = "MAP_LABEL LIKE '"+b+"'"
    arcpy.SelectLayerByAttribute_management(tmpTbl,"ADD_TO_SELECTION",where_clause)
    
results = int(arcpy.GetCount_management(tmpTbl).getOutput(0))
print ("The new selection yielded {0} rows.".format(results))

### Here we can see that by appending the wildcard `%` to our list we ensure that all BEC zones including their subzones and variants are selected from the table. For the final step we can export this selection to a table we can now use for analysis.

In [ ]:
arcpy.CopyRows_management(tmpTbl, "finalBecTable")